# Prepare documents in data/
This file prepares the corpus of documents consisting of the DOI and the abstract from a source

In [19]:
import requests
import pandas as pd
from tqdm import tqdm
from tqdm.gui import tqdm as tqdm_gui
import time
import os


tqdm.pandas()


In [5]:
from reconstruct_abstract import reconstruct_abstract
def get_openalex_data(doi) -> dict:
        """
        Used to retrieve data from the OpenAlex API.
        Arg: takes a DOI as a string without the resolver.
        Return: A dictionary of values.

        Note: oa_abstract is reconstructed from the function reconstruct_abstract(). You will need to install
        https://github.com/poppy-nicolette/Bibliometric_tools/tree/7bcb724c95d9f6a571322076a730736097cf5886/reconstruct_abstract

        Example usage
            doi = "10.1234/example"
            data = get_openalex_data(doi)
            print(data)
        """
        URL = f"https://api.openalex.org/works?filter=doi:{doi}&select=doi,title,abstract_inverted_index"
        try:
            result = requests.get(URL)

            if result.status_code == 200:
                data = result.json()

                # Parse json data into each element:
                try:
                    oa_doi = data['results'][0]['doi'].lstrip('https://doi.org/')
                except:
                    oa_doi = doi
                oa_title = data['results'][0]['title']
                oa_abstract_inverted_index = data['results'][0]['abstract_inverted_index']
                # Reconstruct abstract
                try:
                    oa_abstract = reconstruct_abstract.reconstruct_abstract(oa_abstract_inverted_index)
                except:
                    oa_abstract = "None"

                return {
                    'oa_doi': oa_doi,
                    'oa_title': oa_title,
                    'oa_abstract': oa_abstract,
                    }
            else:
                print(f"Error: Received status code {result.status_code} for DOI {doi}")
                return {'oa_doi':oa_doi,
                        'oa_title':None,
                        'oa_abstract':None}
        except (requests.exceptions.RequestException,IndexError) as e:
            print(f"Request failed for DOI {doi}: {e}")
            return {'oa_doi':oa_doi,
                    'oa_title':None,
                    'oa_abstract':None}
        finally:
            # Sleep so that you are below the 10 per second limit or 100k per day.
            time.sleep(0.11)

#return document for each doi from dictionary 
def prepare_document(x:dict):
    """
    Takes a dictionary of three values as input. 
    Outputs a plain text file
    Input
        dictionary containing oa_doi, oa_title, oa_abstract
    Output
        writes to a text tile
    Returns
        None
    """
    oa_doi = x.get('oa_doi', "None")
    oa_title = x.get('oa_title', "None")
    oa_abstract = x.get('oa_abstract', "None")

    #create file name
    oa_doi_mod = oa_doi.replace("/","_")
    file_name = f"data/{oa_doi_mod}.txt"

    #write to file
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(f"DOI: {oa_doi}\n")
        file.write(f"Title: {oa_title}\n")
        file.write(f"Abstract: {oa_abstract}\n")

In [6]:
doi = "10.3390/su142416618"
prepare_document(get_openalex_data(doi))

## Call .csv file and run functions

In [7]:
# find you a list of DOIs to process
doi_list = pd.read_csv("data/doi_list.csv")
doi_list.info()
print(type(doi_list.iloc[0,0]))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   doi     44 non-null     object
dtypes: object(1)
memory usage: 484.0+ bytes
<class 'str'>


In [8]:
# read the list and work through the list
doi_list['doi'] = doi_list['doi'].astype(str)#check they are all strings

#apply both functions
doi_list['doi'].progress_apply(lambda x: prepare_document(get_openalex_data(x)))
print('done')


  7%|▋         | 3/44 [00:00<00:12,  3.24it/s]

Request failed for DOI 10.18653/v1/D19-1371: list index out of range


 18%|█▊        | 8/44 [00:03<00:17,  2.04it/s]

Request failed for DOI 10.48550/arXiv.2303.17661: list index out of range


 23%|██▎       | 10/44 [00:04<00:15,  2.16it/s]

...

Request failed for DOI 10.48550/arXiv.2402.11782: list index out of range


100%|██████████| 44/44 [00:24<00:00,  1.11it/s]

Request failed for DOI 10.48550/arXiv.2404.13948: list index out of range


100%|██████████| 44/44 [00:24<00:00,  1.76it/s]

done


## Create documents with errorsb - **NOT COMPLETE**
This section takes the same documents from above and introduces errors or characteristics identified in Part 1.
- [ ] JATS tags typical of Crossref
- [ ] multiple languages with multiple unicode blocks
- [ ] MathML tags

In [74]:
# open each txt file

# write txt file with error or characteristic. 

# add in JATS tags at beginning and end.
beginning_pattern = "<jats:title>Abstract</jats:title><jats:p>"
end_pattern = "</jats:p>"

def add_jats_pattern(input_dir:str):

    #open file#read each file in input_dir
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.txt'):
            file_path = os.path.join(input_dir,file_name)

            with open(file_path, 'r') as file:
                content = file.readlines()
                #changing the abstract only 
                abstract = content[2]
                abstract = line.lstrip("Abstract: ")
                abstract = f"Abstract: {beginning_pattern}{abstract}{end_pattern}"# ADD ERRORS HERE
                content[2] = abstract
            # now overwrite the file entirely with variable content
            with open(file_path,"w") as file:
                file.writelines(content)
                print(f"done with {file_name}")
    print(f"done with everything. \n completed {len(os.listdir(input_dir))} files.")




# create another set with multiple unicode blocks translating the title and abstract
#follow same function above but with different patterns

# get chemistry or math papers
# add or remove MathML tags



In [75]:
"""
This applies the jats patterns above onto all text files within a directory.
"""
input_dir = "/Users/poppyriddle/Documents/PhD/Research_proposal/Part_3/part_3_cohere/data copy"
add_jats_pattern(input_dir)

done with 10.48550_arXiv.2406.15154.txt
done with 10.48550_arXiv.2402.01788.txt
done with 10.1590_SciELOPreprints.11205.txt
done with 10.48550_arXiv.2410.04231.txt
done with 10.48550_arXiv.2404.17663.txt
done with 10.1093_jamia_ocae129.txt
done with 10.1007_s11192-023-04923-y.txt
done with 10.48550_arXiv.2401.16359.txt
done with 10.1007_s11192-015-1765-5.txt
done with 10.1162_qss_a_00022.txt
done with 10.48550_arXiv.2404.01985.txt
done with 10.3145_epi.2023.mar.09.txt
done with 10.1162_qss_a_00286.txt
done with 10.5860_crl.86.1.101.txt
done with 10.1007_s11192-022-04367-w.txt
done with 10.1002_leap.1411.txt
done with 10.48550_arXiv.2409.10633.txt
done with 10.31274_b8136f97.ccc3dae4.txt
done with 10.48550_arXiv.2407.17023.txt
done with 10.1145_3626772.3657848.txt
done with 10.48550_arXiv.2502.03627.txt
done with 10.1162_qss_a_00112.txt
done with 10.48550_arXiv.2505.18247.txt
done with 10.48550_arXiv.2303.17661.txt
done with 10.1609_aaai.v38i16.29728.txt
done with 10.48550_arXiv.2406.13